In [ ]:
import torch
import torchvision

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

path = "/content/drive/MyDrive/images1"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root=path, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
len(test_dataset)

64

In [ ]:
import torchvision.models as models

resnet_ = models.resnet50(pretrained=True)

In [ ]:
for param in resnet_.parameters():
  param.requires_grad = False
classes = len(dataset)
resnet_.fc = torch.nn.Linear(in_features=resnet_.fc.in_features, out_features=classes)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet_.fc.parameters(), lr=0.0001)

In [25]:
num_epochs = 10

for epoch in range(num_epochs):
    resnet_.train()
    loss_running = 0
    correct = 0
    total = 0

    for img, label in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        forward = resnet_(img)
        # No need to modify the label tensor here
        loss = criterion(forward, label)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(forward.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        loss_running += loss.item()

    training_loss = loss_running / len(train_loader)
    accuracy = correct / total

    # Evaluation on test set
    resnet_.eval()
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for images, labels in test_loader:
            outputs = resnet_(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_accuracy = correct_test / total_test
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {training_loss:.4f}, Train Accuracy: {100 * accuracy:.2f}%, Test Accuracy: {100 * test_accuracy:.2f}%')


Epoch 1/10, Train Loss: 0.6628, Train Accuracy: 60.94%, Test Accuracy: 56.25%
Epoch 2/10, Train Loss: 0.6621, Train Accuracy: 60.55%, Test Accuracy: 78.12%
Epoch 3/10, Train Loss: 0.6400, Train Accuracy: 61.33%, Test Accuracy: 65.62%
Epoch 4/10, Train Loss: 0.5968, Train Accuracy: 75.00%, Test Accuracy: 81.25%
Epoch 5/10, Train Loss: 0.5834, Train Accuracy: 78.91%, Test Accuracy: 70.31%
Epoch 6/10, Train Loss: 0.5610, Train Accuracy: 74.61%, Test Accuracy: 81.25%
Epoch 7/10, Train Loss: 0.5403, Train Accuracy: 81.25%, Test Accuracy: 89.06%
Epoch 8/10, Train Loss: 0.5268, Train Accuracy: 79.69%, Test Accuracy: 85.94%
Epoch 9/10, Train Loss: 0.5057, Train Accuracy: 85.55%, Test Accuracy: 87.50%
Epoch 10/10, Train Loss: 0.4980, Train Accuracy: 81.64%, Test Accuracy: 87.50%


In [26]:
torch.save(resnet_.state_dict(), "resnet_face_recognition.pth")

In [27]:
dataset.classes

['messi', 'ronaldo']